In [1]:
import tensorflow as tf
import pandas as pd
import keras
train = pd.read_csv("dataset_train_set.csv")
test = pd.read_csv("dataset_dev_set.csv")

Using TensorFlow backend.


In [2]:
X_train = train.drop('target',axis=1)
y_train = train['target']
X_test = test.drop('target',axis=1)
y_test = test['target']

In [3]:
X_train = X_train[['aPosLS','hPosLS','hGD5','aSpent','aPtsPG','hSPG','hGCPG','aPos','aSTPG','aGSPG','hSTPG','aSPG','aGD7','hPos','aGD5','hPtsPG','hSpent','aPTSPG7','hGD7']].copy()
X_test = X_test[['aPosLS','hPosLS','hGD5','aSpent','aPtsPG','hSPG','hGCPG','aPos','aSTPG','aGSPG','hSTPG','aSPG','aGD7','hPos','aGD5','hPtsPG','hSpent','aPTSPG7','hGD7']].copy()

In [4]:
X_train.head()

,aPosLS,hPosLS,hGD5,aSpent,aPtsPG,hSPG,hGCPG,aPos,aSTPG,aGSPG,hSTPG,aSPG,aGD7,hPos,aGD5,hPtsPG,hSpent,aPTSPG7,hGD7
0,18,18,0.0,18.07,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,18.72,0.0,0.0
1,9,5,0.0,8.92,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,47.15,0.0,0.0
2,12,14,0.0,29.52,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,17.10,0.0,0.0
3,15,16,0.0,0.00,0.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,11.99,0.0,0.0
4,13,3,0.0,31.03,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,47.84,0.0,0.0


In [5]:
print(X_train.shape)
X_train_norm = tf.keras.utils.normalize(X_train.values,axis=1)
X_test_norm = tf.keras.utils.normalize(X_test.values,axis=1)
X_train_norm[1000]

(4180, 19)


array([ 0.20521545,  0.24625854, -0.00820862,  0.16622452,  0.05294559,
        0.51131482,  0.03940137,  0.49251709,  0.21720004,  0.05294559,
        0.2685039 ,  0.4446198 ,  0.04104309,  0.16417236,  0.03283447,
        0.07018369,  0.        ,  0.08208618,  0.00586916])

In [6]:
y_train.value_counts()

2    1971
0    1120
1    1089
Name: target, dtype: int64

In [7]:
whome = 1
wdraw = 1971/1089
waway = 1971/1120
print("{},{},{}".format(whome,wdraw,waway))

1,1.8099173553719008,1.7598214285714286


In [8]:
class_weights = {0:1.7598,
                 1:1.8099,
                 2:1}

In [9]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(3, activation = tf.nn.softmax))

In [10]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [11]:
model.fit(X_train_norm, y_train.values, epochs=50, validation_data=(X_test_norm,y_test), class_weight=class_weights)

Train on 4180 samples, validate on 1520 samples
Epoch 1/50
4180/4180 [==============================] - 1s 164us/step - loss: 1.0922 - acc: 0.3189 - val_loss: 1.0820 - val_acc: 0.4263
Epoch 2/50
4180/4180 [==============================] - 0s 78us/step - loss: 1.0749 - acc: 0.4108 - val_loss: 1.0541 - val_acc: 0.4789
Epoch 3/50
4180/4180 [==============================] - 0s 61us/step - loss: 1.0542 - acc: 0.4634 - val_loss: 1.0221 - val_acc: 0.4980
Epoch 4/50
4180/4180 [==============================] - 0s 62us/step - loss: 1.0422 - acc: 0.4653 - val_loss: 1.0078 - val_acc: 0.5132
Epoch 5/50
4180/4180 [==============================] - 0s 64us/step - loss: 1.0378 - acc: 0.4627 - val_loss: 0.9997 - val_acc: 0.5204
Epoch 6/50
4180/4180 [==============================] - 0s 66us/step - loss: 1.0341 - acc: 0.4780 - val_loss: 1.0056 - val_acc: 0.5138
Epoch 7/50
4180/4180 [==============================] - 0s 62us/step - loss: 1.0327 - acc: 0.4701 - val_loss: 1.0016 - val_acc: 0.5217
Epoch 

In [12]:
val_loss, val_acc = model.evaluate(X_test_norm, y_test)
print(val_loss, val_acc)

1520/1520 [==============================] - 0s 53us/step
0.9893782465081465 0.5269736842105263


In [13]:
predictions = model.predict_classes(X_test_norm)
preds = list(predictions)
pd.Series(list(predictions)).value_counts()

2    688
0    502
1    330
dtype: int64

In [14]:
predictions[0]

2

In [15]:
correct_pred_dict = {0:0,1:0,2:0}
for i in range(len(preds)):
    pred = preds[i]
    real = y_test[i]
    if pred == real:
        correct_pred_dict[pred] += 1
correct_pred_dict

{0: 258, 1: 109, 2: 434}

In [16]:
outcomes_incorrectly_caught_dict = {0:0,1:0,2:0}
for i in range(len(preds)):
    pred = preds[i]
    real = y_test[i]
    if pred != real:
        outcomes_incorrectly_caught_dict[real] += 1
outcomes_incorrectly_caught_dict

{0: 202, 1: 263, 2: 254}